In [ ]:
#mac
!brew install portaudio 
!pip install pyaudio
!pip install --upgrade aylien-apiclient
!pip install gTTS

In [ ]:
#pc
!python -m pip install pyaudio
!pip install --upgrade aylien-apiclient
!pip install gTTS

In [1]:
import time
import requests
from bs4 import BeautifulSoup
from aylienapiclient import textapi
import pyaudio
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
import subprocess
from gtts import gTTS
from sys import exit
import warnings
warnings.filterwarnings('ignore')

def tts(text):
    audio_file = "text.mp3"
    tts = gTTS(text=text, lang="en")
    tts.save(audio_file)
    return_code = subprocess.call(["afplay", audio_file])

def play_music(song_name):

    import wave
    # set desired values in seconds
    start=0
    length=10
    # open wave file
    wave=wave.open(song_name, 'rb')
    # initialize audio
    py_audio=pyaudio.PyAudio()
    stream=py_audio.open(format=py_audio.get_format_from_width(wave.getsampwidth()),
                           channels=wave.getnchannels(),
                           rate=wave.getframerate(),
                           output=True)

    # skip unwanted frames
    n_frames=int(start * wave.getframerate())
    wave.setpos(n_frames)

    # write desired frames to audio buffer
    n_frames=int(length * wave.getframerate())
    frames=wave.readframes(n_frames)
    stream.write(frames)

    # close and terminate everything properly
    wave.close()
    stream.close()
    py_audio.terminate()

def get_html(url):
    # Get html from url
    response=requests.get(url)
    return response.text

def extract_info(html):
    #how we pull data from webpage
    html=webpage
    soup=BeautifulSoup(html, "lxml")
    title=soup.select('span.titletext')
    return title   

def google_geocode(location):
    #gets location
    params = {'address' : location}
    url = 'http://maps.googleapis.com/maps/api/geocode/json'
    response=requests.get(url, params = params)
    geodata=response.json()
    return geodata

def darksky_weather(coords):
    #gets weather data using coords
    key='e27a586aeaa37333949a012de551896f' 
    url='https://api.darksky.net/forecast/%s/%f,%f' % (key, coords['lat'], coords['lng'])
    response=requests.get(url)
    weather=response.json()
    return weather

def GetHourlyWeather():
    #get hourly weather
    responce = requests.get('https://api.darksky.net/forecast/e27a586aeaa37333949a012de551896f/%f,%f' % (coords['lat'], coords['lng']))
    data = responce.json()
    hrly = data['hourly']['data']
    return hrly

#pull headlines
try:
    count=0
    text=''
    webpage = get_html('https://news.google.com')
    headlines = extract_info(webpage)
    print('Daily Morning Briefing!\n')
    print('Headlines:')
except:
    print('No internet, try again later')
    exit()

for head in headlines:
    count+=1
    if count%3==0 and count<31: #print every 3 so not repetitive stop and only top 10
        print(head.text)
        text=(text+' ,,'+head.text)

    #print(count)

#sentement analysis
try:
    client = textapi.Client("62c33dff", "48f2b35bfa2f83d174f9ee7c1ccca651")
    sentiment = client.Sentiment({'text': text})
    polarity_confidence=sentiment['polarity_confidence']*100
    polarity=sentiment['polarity']
    confidence_statement=("We are %.1f%% sure today's news is %s." %(polarity_confidence, polarity))
    print(confidence_statement)
    text=(text+' ,,'+confidence_statement)
except:
    confidence_statement=('We were not able to access the server. Please try again later.')
    print(confidence_statement)
    text=(text+' ,,'+confidence_statement)

talk=input('\nWould you like to hear the headlines outloud?: ')
talk=talk.lower()
if 'y' in talk:
    tts(text)

#choosing what music to play
tts('Here is your song for the day!')
if polarity=='positive':
    play_music('happy.wav')
elif polarity=='negative':
    play_music('hurt.wav')
else:
    play_music('theone.wav')


#Hourly Weather Part
try:
    location = input("\nWhere are you located? (eg. Syracuse, NY) : ")
    geodata = google_geocode(location)
    coords =geodata['results'][0]['geometry']['location']
    weather = darksky_weather(coords)
    current = weather['currently']

    forecast = GetHourlyWeather()
    forecast_df = pd.DataFrame(forecast)

    time = forecast_df['time']
    readable_time = pd.to_datetime(time, unit='s')
    forecast_df['time'] = readable_time

    hourly_temps = forecast_df[['time', 'apparentTemperature']].head(24)
except IndexError:
    print('That is not a vaild location, try again')
    exit()
#plotting weather
hourly_temps.iplot(kind = 'line', x = 'time', y = 'apparentTemperature', title='Temperature by Hour in %s' % (location), xTitle='Hours from now',yTitle='Temperature (°F)')



Daily Morning Briefing!

Headlines:
House to vote Thursday on Obamacare repeal bill
FBI director says he feels 'mildly nauseous' about possibility he affected election, but has no regrets
Trump to ease enforcement of limits on church political activity
White House aims for Thursday signing of religious liberty executive order
This Is Why That Google Doc Spear Phishing Email Tricked So Many People
Google Docs users hit with sophisticated phishing attack
Obama unveils vision for presidential library in Chicago
Sweet: Obama Center design is splashy with welcoming streetscape
French election: Macron 'most convincing' in final debate
On World Press Freedom Day, student shares her opinion on why a free press matters
We are 89.6% sure today's news is neutral.

Would you like to hear the headlines outloud?: yes

Where are you located? (eg. Syracuse, NY) : syracuse
